In [2]:
import cv2
import os
import numpy as np
from PIL import Image
import tkinter as tk
from tkinter import messagebox
import openpyxl
from datetime import datetime

# Ensure Excel files exist
def init_excels():
    if not os.path.exists("attendance.xlsx"):
        wb = openpyxl.Workbook()
        ws = wb.active
        ws.append(["Registration Number", "Name", "Time"])
        wb.save("attendance.xlsx")
    
    if not os.path.exists("users.xlsx"):
        wb = openpyxl.Workbook()
        ws = wb.active
        ws.append(["ID", "Registration Number", "Name"])
        wb.save("users.xlsx")

def save_attendance(reg_no, name):
    wb = openpyxl.load_workbook("attendance.xlsx")
    ws = wb.active
    ws.append([reg_no, name, datetime.now().strftime("%Y-%m-%d %H:%M:%S")])
    wb.save("attendance.xlsx")

def save_user_mapping(user_id, reg_no, name):
    wb = openpyxl.load_workbook("users.xlsx")
    ws = wb.active
    for row in ws.iter_rows(min_row=2, values_only=True):
        if row[1] == reg_no:
            return
    ws.append([user_id, reg_no, name])
    wb.save("users.xlsx")

def generate_dataset(reg_no, name):
    face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
        if len(faces) == 0:
            return None
        for (x, y, w, h) in faces:
            return img[y:y+h, x:x+w]

    cap = cv2.VideoCapture(0)
    os.makedirs("data", exist_ok=True)

    existing_ids = set()
    for f in os.listdir("data"):
        if f.endswith(".jpg"):
            try:
                user_id = int(f.split(".")[1])
                existing_ids.add(user_id)
            except:
                pass
    user_id = max(existing_ids, default=0) + 1

    img_id = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            continue
        cropped = face_cropped(frame)
        if cropped is not None:
            img_id += 1
            face = cv2.resize(cropped, (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            file_name_path = f"data/user.{user_id}.{img_id}.jpg"
            cv2.imwrite(file_name_path, face)
            cv2.putText(face, str(img_id), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            cv2.imshow("Training Face", face)
        if cv2.waitKey(1) == 13 or img_id == 200:
            break

    cap.release()
    cv2.destroyAllWindows()
    save_user_mapping(user_id, reg_no, name)
    messagebox.showinfo("Success", f"Collected 200 images for {name} ({reg_no})")

def train_classifier():
    path = [os.path.join("data", f) for f in os.listdir("data") if f.endswith(".jpg")]
    faces = []
    ids = []

    for image in path:
        img = Image.open(image).convert('L')
        imageNp = np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split(".")[1])
        faces.append(imageNp)
        ids.append(id)

    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces, np.array(ids))
    clf.write("classifier.xml")
    messagebox.showinfo("Training Complete", "Model saved as classifier.xml")

def recognize_faces():
    if not os.path.exists("classifier.xml") or not os.path.exists("users.xlsx"):
        messagebox.showerror("Error", "No trained model or user data found.")
        return

    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.read("classifier.xml")
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

    wb = openpyxl.load_workbook("users.xlsx")
    ws = wb.active
    id_map = {int(row[0]): (row[1], row[2]) for row in ws.iter_rows(min_row=2, values_only=True)}

    cap = cv2.VideoCapture(0)
    recognized_users = set()

    while True:
        ret, frame = cap.read()
        if not ret:
            continue

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in faces:
            roi = gray[y:y+h, x:x+w]
            id, pred = clf.predict(roi)
            confidence = int(100 * (1 - pred / 300))

            if confidence > 70 and id in id_map:
                reg_no, name = id_map[id]
                label = f"{name} ({reg_no})"
                recognized_users.add((reg_no, name))
                cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
            else:
                cv2.putText(frame, "UNKNOWN", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 255, 255), 2)

        cv2.imshow("Face Recognition", frame)
        if cv2.waitKey(1) == 13:  # Enter key to exit
            break

    cap.release()
    cv2.destroyAllWindows()

    for reg_no, name in recognized_users:
        save_attendance(reg_no, name)

    messagebox.showinfo("Done", "Attendance marked for detected users.")

# GUI with additional "Train Classifier" button
def start_gui():
    def on_train_images():
        reg_no = reg_entry.get()
        name = name_entry.get()
        if not reg_no or not name:
            messagebox.showerror("Input Error", "Please fill in both fields.")
            return
        generate_dataset(reg_no, name)

    def on_train_model():
        train_classifier()

    root = tk.Tk()
    root.title("Smart Attendance System")
    root.geometry("400x320")
    root.resizable(False, False)

    tk.Label(root, text="Registration Number").pack(pady=5)
    reg_entry = tk.Entry(root, width=30)
    reg_entry.pack()

    tk.Label(root, text="Name").pack(pady=5)
    name_entry = tk.Entry(root, width=30)
    name_entry.pack()

    tk.Button(root, text="Train Images", command=on_train_images, bg="lightblue", width=25).pack(pady=10)
    tk.Button(root, text="Train Classifier", command=on_train_model, bg="orange", width=25).pack(pady=5)
    tk.Button(root, text="Detect and Mark Attendance", command=recognize_faces, bg="lightgreen", width=25).pack(pady=10)

    root.mainloop()

# Entry point
if __name__ == "__main__":
    init_excels()
    start_gui() 
